In [1]:
# Procesamiento de datos
import pandas as pd
# Funciones algebráicas
import numpy as np
# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
## Estilos
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')
# Contador
from collections import Counter
# Métodos de iteración
from itertools import product
# Machine Learning
## Cross Validation
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
## Separación de datos de entrenamiento
from sklearn.model_selection import train_test_split
## Preprocesamiento de variables
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder, RobustScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer
# Modelos de clasificacion
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

## Modelo de máquina de soporte vectorial
from sklearn.svm import SVC
## Clasificador de árbol de decisión
from sklearn.tree import DecisionTreeClassifier
## Clasificador de bosques aleatorios
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
## Naive Bayes
from sklearn.naive_bayes import GaussianNB
## Regresión Logística
from sklearn.linear_model import LogisticRegression
## Muestreado para balancear datos
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
## Métricas
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc, roc_auc_score, ConfusionMatrixDisplay, f1_score, make_scorer, precision_score, recall_score, log_loss
## Ajuste de hiperparámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# módulo que proporciona soporte en tiempo de ejecución para sugerencias de tipo
from sklearn import datasets
from typing import Dict, Iterable, Any
# Desactivar los avisos
import warnings
warnings.filterwarnings('ignore')
# Importar los datos de yahoo
import yfinance as yf
# Librería para plotear mejor datos financieros
import cufflinks as cf
#cf.set_config_file(offline = True)

He intentado escoger acciones con poca correlación. La explicación de ellas es:

AAPL: Apple Inc., uno de los fabricantes más grandes de tecnología de consumo, conocido por productos como el iPhone, el iPad y las MacBooks.

RELIANCE.NS: Reliance Industries Limited, una conglomerado multinacional indio que tiene negocios en diversos sectores como energía, petroquímicos, textiles, recursos naturales, y telecomunicaciones. El sufijo ".NS" indica que la acción se negocia en la National Stock Exchange of India.

BHP.AX: BHP Group Limited (anteriormente BHP Billiton), una de las compañías mineras y de recursos más grandes del mundo. El sufijo ".AX" indica que la acción se negocia en la Australian Securities Exchange.

SAN.MC: Banco Santander, S.A., un banco multinacional español y una de las mayores instituciones bancarias de la zona euro. El sufijo ".MC" indica que la acción se negocia en la Bolsa de Madrid.

REP.MC: Repsol, S.A., una empresa energética y petroquímica española. Al igual que SAN.MC, el sufijo ".MC" indica que la acción se negocia en la Bolsa de Madrid.

IBE.MC: Iberdrola, S.A., una empresa española de energía, una de las mayores productoras de energía renovable. También cotiza en la Bolsa de Madrid.

TEF.MC: Telefónica, S.A., una compañía multinacional de telecomunicaciones con sede en España. También cotiza en la Bolsa de Madrid.

Acciones con las que trabajaré: 

["AAPL", "RELIANCE.NS", "BHP.AX", "SAN.MC", "REP.MC", "IBE.MC", "TEF.MC"]

In [2]:
df = pd.read_csv("acciones_preprocesado.csv")
# quito los que devuelve 0 la dirección ya que no me calcula bien las métricas de los modelos
df = df[df.direction != 0]
df.reset_index(inplace=True, drop=True)
df

,accion,date,price,price_lag1,high_lag1,low_lag1,direction,lag1,lag2,lag3,...,roll_high,year,day_of_year,week_of_year,sin_month,cos_month,sin_day,cos_day,sin_day_of_week,cos_day_of_week
0,AAPL,2010-10-26,9.314108,9.337992,11.128571,11.015714,-1.0,0.004446,-0.006645,-0.003258,...,11.392857,2010,299,43,-8.660254e-01,0.500000,-0.848644,0.528964,0.781831,0.623490
1,AAPL,2010-10-27,9.307454,9.314108,11.062143,10.916071,-1.0,-0.002561,0.004446,-0.006645,...,11.392857,2010,300,43,-8.660254e-01,0.500000,-0.724793,0.688967,0.974928,-0.222521
2,AAPL,2010-10-28,9.229143,9.307454,11.067857,10.914286,-1.0,-0.000715,-0.002561,0.004446,...,11.392857,2010,301,43,-8.660254e-01,0.500000,-0.571268,0.820763,0.433884,-0.900969
3,AAPL,2010-10-29,9.100339,9.229143,11.000000,10.746429,-1.0,-0.008449,-0.000715,-0.002561,...,11.392857,2010,302,43,-8.660254e-01,0.500000,-0.394356,0.918958,-0.433884,-0.900969
4,AAPL,2010-11-01,9.197092,9.100339,10.924286,10.745357,1.0,-0.014055,-0.008449,-0.000715,...,11.392857,2010,305,44,-5.000000e-01,0.866025,0.201299,0.979530,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16674,TEF.MC,2023-12-21,3.492111,3.546088,3.820000,3.646000,-1.0,0.031477,-0.007545,-0.009145,...,4.075000,2023,355,51,-2.449294e-16,1.000000,-0.897805,-0.440394,0.433884,-0.900969
16675,TEF.MC,2023-12-22,3.454520,3.492111,3.685000,3.615000,-1.0,-0.015339,0.031477,-0.007545,...,4.075000,2023,356,51,-2.449294e-16,1.000000,-0.968077,-0.250653,-0.433884,-0.900969
16676,TEF.MC,2023-12-27,3.441990,3.454520,3.650000,3.571000,-1.0,-0.010823,-0.015339,0.031477,...,4.075000,2023,361,52,-2.449294e-16,1.000000,-0.724793,0.688967,0.974928,-0.222521
16677,TEF.MC,2023-12-28,3.412110,3.441990,3.590000,3.555000,-1.0,-0.003634,-0.010823,-0.015339,...,4.075000,2023,362,52,-2.449294e-16,1.000000,-0.571268,0.820763,0.433884,-0.900969


Lo primero que voy a hacer es transformar la fecha para poder meterla como variable numérica 

Ahora voy a separar los data frames en:
1.- variables categóricas para hacerle onehotencoding
2.- variables numéricas para aplicarle escaladores
3.- variables seno y cosenoidales que no se transforman ya más
4.- variable objetivo (dirección)

Analizar acción Iberdrola 

In [3]:
df_filtered = df[df["accion"] == "IBE.MC"].copy()

# Asegurarse de que la columna 'date' está en el formato de fecha
df_filtered['date'] = pd.to_datetime(df_filtered['date'])

# Ordenar el DataFrame por fecha
df_filtered = df_filtered.sort_values(by='date')

# Establecer la columna 'date' como índice
df_filtered.set_index('date', inplace=True)

# Plotear el precio a lo largo del tiempo usando cufflinks
df_filtered['price'].iplot(
    title='Precio de la acción Iberdrola a lo largo del tiempo',
    xTitle='Fecha',
    yTitle='Precio'
)

Vamos a ver el rendimiento del modelo general cuando IBERDROLA tiene tendencias alcista, bajista, sin tendencia definida
- Bajista: 1/11/2007 a 1/3/2009
- Alcista: 1/11/2018 a 1/02/2020
- Sin tendencia definida: 1/03/2009 a 1/02/2012